In [1]:
%matplotlib inline

import torch
import matplotlib.pyplot as plt
import torchvision
from torchvision import transforms
import torch.nn.functional as F
import torch.optim as optim
from torchsummary import summary

In [21]:
# Hyper parameters
num_epochs = 50
batchsize = 16
batchsize_test = 16
learning_rate = 0.001
momentum = 0.5
log_interval = 20
TRAIN_DATA_PATH = "../Data/CATS_DOGS/train"
TEST_DATA_PATH = "../Data/CATS_DOGS/test"

TRANSFORM_IMG = transforms.Compose([
    transforms.Resize(150),
    transforms.CenterCrop(150),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225] )
    ])

train_data = torchvision.datasets.ImageFolder(root=TRAIN_DATA_PATH, transform=TRANSFORM_IMG)
train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=batchsize, shuffle=True,  num_workers=4)

test_data = torchvision.datasets.ImageFolder(root=TEST_DATA_PATH, transform=TRANSFORM_IMG)
test_data_loader  = torch.utils.data.DataLoader(test_data, batch_size=batchsize_test, shuffle=True, num_workers=4) 



In [22]:
class Network(torch.nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.conv1 = torch.nn.Conv2d(3, 20, kernel_size=3)
        self.conv2 = torch.nn.Conv2d(20, 32, kernel_size=3)
        self.conv3 = torch.nn.Conv2d(32, 64, kernel_size=3)
        self.fc1 = torch.nn.Linear((64*17*17), 50)
        self.fc2 = torch.nn.Linear(50, 1)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = F.relu(F.max_pool2d(self.conv3(x), 2))
        x = x.view(-1, (64*17*17))
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.sigmoid(x)

In [23]:
#Select device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#Network model
network = Network()
network.to(device)

#Stochastic gradient decent
optimizer = optim.SGD(network.parameters(), lr=learning_rate,
                      momentum=momentum)

criterion = torch.nn.BCELoss()

In [24]:
summary(network, (3, 150, 150))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 20, 148, 148]             560
            Conv2d-2           [-1, 32, 72, 72]           5,792
            Conv2d-3           [-1, 64, 34, 34]          18,496
            Linear-4                   [-1, 50]         924,850
            Linear-5                    [-1, 1]              51
Total params: 949,749
Trainable params: 949,749
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.26
Forward/backward pass size (MB): 5.17
Params size (MB): 3.62
Estimated Total Size (MB): 9.05
----------------------------------------------------------------


In [6]:
##Just for testign
test = iter(train_data_loader)
img, target= next(test)

#out = network(img)
#test_loss += F.binary_cross_entropy(output, target).item()
#plt.imshow(img[9][0])

#just for testing
conv1 = torch.nn.Conv2d(3, 20, kernel_size=3)
conv2 = torch.nn.Conv2d(20, 32, kernel_size=3)
conv3 = torch.nn.Conv2d(32, 64, kernel_size=3)
drop = torch.nn.Dropout2d()
fc1 = torch.nn.Linear((64*17*17), 50)
fc2 = torch.nn.Linear(50, 1)

x = F.relu(F.max_pool2d(conv1(img.to(device)), 2))
x = F.relu(F.max_pool2d(conv2(x), 2))
x = F.relu(F.max_pool2d(conv3(x), 2))
x = x.view(-1, (64*17*17))
x = F.relu(fc1(x))
x = F.dropout(x)
x = fc2(x)
x = F.sigmoid(x)

x.shape

/opt/conda/lib/python3.8/site-packages/torch/nn/functional.py:1944: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


torch.Size([16, 1])

In [12]:
target.view(-1, 1).shape

torch.Size([64, 1])

In [25]:
train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(train_data_loader.dataset) for i in range(num_epochs + 1)]

In [29]:
def test():
    #Sets the module in evaluation mode
    network.eval()
    test_loss = 0
    correct = 0
    
    #dont update dynamic computation graph
    with torch.no_grad():
        #for every example in test
        for data, target in test_data_loader:
            #evaluate the model
            output = network(data.to(device))
            
            #acumulate the loss
            test_loss += F.binary_cross_entropy(output, target.view(-1, 1).float().to(device)).item()
            
            
            output.to('cpu')
            target.to('cpu')
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).sum()
        
    test_loss /= len(test_data_loader.dataset)
    test_losses.append(test_loss)
    print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_data_loader.dataset),
    100. * correct / len(test_data_loader.dataset)))

In [26]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [30]:
#train

#test()

for epoch in range(1, num_epochs+1):
    
    for batch_idx, (data, target) in enumerate(train_data_loader):
        
        network.train()
        
        #print(f'b {batch}')
        
        #forward pass
        out = network(data.to(device))
        
        #Use negative log likelihood loss.
        loss = criterion(out, target.view(-1, 1).float().to(device))
        
        
        #with this gradients are calculated
        loss.backward()
    
        #update gradients
        optimizer.step()

        #Display iteration statistics
        if batch_idx % log_interval == 0:
            
            #print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch, batch_idx * len(data), len(train_data_loader.dataset),100. * batch_idx / len(train_data_loader), loss.item()))
        
#test()
    
    

Train Epoch: 1 [0/18743 (0%)]	Loss: 0.710319
Train Epoch: 1 [320/18743 (2%)]	Loss: 0.696560
Train Epoch: 1 [640/18743 (3%)]	Loss: 0.693261
Train Epoch: 1 [960/18743 (5%)]	Loss: 0.700269
Train Epoch: 1 [1280/18743 (7%)]	Loss: 0.652782
Train Epoch: 1 [1600/18743 (9%)]	Loss: 0.826878
Train Epoch: 1 [1920/18743 (10%)]	Loss: 0.692074
Train Epoch: 1 [2240/18743 (12%)]	Loss: 0.717464
Train Epoch: 1 [2560/18743 (14%)]	Loss: 0.676724
Train Epoch: 1 [2880/18743 (15%)]	Loss: 0.766508
Train Epoch: 1 [3200/18743 (17%)]	Loss: 0.654221
Train Epoch: 1 [3520/18743 (19%)]	Loss: 0.682928
Train Epoch: 1 [3840/18743 (20%)]	Loss: 0.670522
Train Epoch: 1 [4160/18743 (22%)]	Loss: 0.687168
Train Epoch: 1 [4480/18743 (24%)]	Loss: 0.667674


/opt/conda/lib/python3.8/site-packages/PIL/TiffImagePlugin.py:822: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Train Epoch: 1 [4800/18743 (26%)]	Loss: 0.688506
Train Epoch: 1 [5120/18743 (27%)]	Loss: 0.667993
Train Epoch: 1 [5440/18743 (29%)]	Loss: 0.821945
Train Epoch: 1 [5760/18743 (31%)]	Loss: 0.708077
Train Epoch: 1 [6080/18743 (32%)]	Loss: 0.700675
Train Epoch: 1 [6400/18743 (34%)]	Loss: 0.671224
Train Epoch: 1 [6720/18743 (36%)]	Loss: 0.740394
Train Epoch: 1 [7040/18743 (38%)]	Loss: 0.690657
Train Epoch: 1 [7360/18743 (39%)]	Loss: 0.726450
Train Epoch: 1 [7680/18743 (41%)]	Loss: 0.717839
Train Epoch: 1 [8000/18743 (43%)]	Loss: 0.695850
Train Epoch: 1 [8320/18743 (44%)]	Loss: 0.706911
Train Epoch: 1 [8640/18743 (46%)]	Loss: 0.689392
Train Epoch: 1 [8960/18743 (48%)]	Loss: 0.675463
Train Epoch: 1 [9280/18743 (49%)]	Loss: 0.650744
Train Epoch: 1 [9600/18743 (51%)]	Loss: 0.904109
Train Epoch: 1 [9920/18743 (53%)]	Loss: 0.659795
Train Epoch: 1 [10240/18743 (55%)]	Loss: 0.663902
Train Epoch: 1 [10560/18743 (56%)]	Loss: 0.699459
Train Epoch: 1 [10880/18743 (58%)]	Loss: 0.629198
Train Epoch: 1 [1

/opt/conda/lib/python3.8/site-packages/PIL/TiffImagePlugin.py:822: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Train Epoch: 2 [10240/18743 (55%)]	Loss: 0.691308
Train Epoch: 2 [10560/18743 (56%)]	Loss: 0.687626
Train Epoch: 2 [10880/18743 (58%)]	Loss: 0.717608
Train Epoch: 2 [11200/18743 (60%)]	Loss: 0.675094
Train Epoch: 2 [11520/18743 (61%)]	Loss: 0.695841
Train Epoch: 2 [11840/18743 (63%)]	Loss: 0.694104
Train Epoch: 2 [12160/18743 (65%)]	Loss: 0.690776
Train Epoch: 2 [12480/18743 (67%)]	Loss: 0.727015
Train Epoch: 2 [12800/18743 (68%)]	Loss: 0.718022
Train Epoch: 2 [13120/18743 (70%)]	Loss: 0.629487
Train Epoch: 2 [13440/18743 (72%)]	Loss: 0.703487
Train Epoch: 2 [13760/18743 (73%)]	Loss: 0.656006
Train Epoch: 2 [14080/18743 (75%)]	Loss: 0.718691
Train Epoch: 2 [14400/18743 (77%)]	Loss: 0.693908
Train Epoch: 2 [14720/18743 (78%)]	Loss: 0.695499
Train Epoch: 2 [15040/18743 (80%)]	Loss: 0.715042
Train Epoch: 2 [15360/18743 (82%)]	Loss: 0.717664
Train Epoch: 2 [15680/18743 (84%)]	Loss: 0.728823
Train Epoch: 2 [16000/18743 (85%)]	Loss: 0.757763
Train Epoch: 2 [16320/18743 (87%)]	Loss: 0.727714


...

Train Epoch: 48 [320/18743 (2%)]	Loss: 50.000000
Train Epoch: 48 [640/18743 (3%)]	Loss: 37.500000
Train Epoch: 48 [960/18743 (5%)]	Loss: 62.500000
Train Epoch: 48 [1280/18743 (7%)]	Loss: 25.000000
Train Epoch: 48 [1600/18743 (9%)]	Loss: 56.250000
Train Epoch: 48 [1920/18743 (10%)]	Loss: 37.500000
Train Epoch: 48 [2240/18743 (12%)]	Loss: 50.000000
Train Epoch: 48 [2560/18743 (14%)]	Loss: 50.000000
Train Epoch: 48 [2880/18743 (15%)]	Loss: 31.250000
Train Epoch: 48 [3200/18743 (17%)]	Loss: 75.000000
Train Epoch: 48 [3520/18743 (19%)]	Loss: 25.000000
Train Epoch: 48 [3840/18743 (20%)]	Loss: 37.500000
Train Epoch: 48 [4160/18743 (22%)]	Loss: 37.500000
Train Epoch: 48 [4480/18743 (24%)]	Loss: 37.500000
Train Epoch: 48 [4800/18743 (26%)]	Loss: 68.750000
Train Epoch: 48 [5120/18743 (27%)]	Loss: 56.250000
Train Epoch: 48 [5440/18743 (29%)]	Loss: 75.000000
Train Epoch: 48 [5760/18743 (31%)]	Loss: 37.500000
Train Epoch: 48 [6080/18743 (32%)]	Loss: 68.750000
Train Epoch: 48 [6400/18743 (34%)]	Loss

/opt/conda/lib/python3.8/site-packages/PIL/TiffImagePlugin.py:822: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Train Epoch: 49 [1600/18743 (9%)]	Loss: 43.750000
Train Epoch: 49 [1920/18743 (10%)]	Loss: 43.750000
Train Epoch: 49 [2240/18743 (12%)]	Loss: 68.750000
Train Epoch: 49 [2560/18743 (14%)]	Loss: 50.000000
Train Epoch: 49 [2880/18743 (15%)]	Loss: 43.750000
Train Epoch: 49 [3200/18743 (17%)]	Loss: 62.500000
Train Epoch: 49 [3520/18743 (19%)]	Loss: 50.000000
Train Epoch: 49 [3840/18743 (20%)]	Loss: 50.000000
Train Epoch: 49 [4160/18743 (22%)]	Loss: 50.000000
Train Epoch: 49 [4480/18743 (24%)]	Loss: 62.500000
Train Epoch: 49 [4800/18743 (26%)]	Loss: 62.500000
Train Epoch: 49 [5120/18743 (27%)]	Loss: 50.000000
Train Epoch: 49 [5440/18743 (29%)]	Loss: 75.000000
Train Epoch: 49 [5760/18743 (31%)]	Loss: 43.750000
Train Epoch: 49 [6080/18743 (32%)]	Loss: 43.750000
Train Epoch: 49 [6400/18743 (34%)]	Loss: 31.250000
Train Epoch: 49 [6720/18743 (36%)]	Loss: 56.250000
Train Epoch: 49 [7040/18743 (38%)]	Loss: 75.000000
Train Epoch: 49 [7360/18743 (39%)]	Loss: 25.000000
Train Epoch: 49 [7680/18743 (41%

/opt/conda/lib/python3.8/site-packages/PIL/TiffImagePlugin.py:822: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Train Epoch: 50 [6400/18743 (34%)]	Loss: 56.250000
Train Epoch: 50 [6720/18743 (36%)]	Loss: 56.250000
Train Epoch: 50 [7040/18743 (38%)]	Loss: 68.750000
Train Epoch: 50 [7360/18743 (39%)]	Loss: 37.500000
Train Epoch: 50 [7680/18743 (41%)]	Loss: 68.750000
Train Epoch: 50 [8000/18743 (43%)]	Loss: 68.750000
Train Epoch: 50 [8320/18743 (44%)]	Loss: 81.250000
Train Epoch: 50 [8640/18743 (46%)]	Loss: 75.000000
Train Epoch: 50 [8960/18743 (48%)]	Loss: 81.250000
Train Epoch: 50 [9280/18743 (49%)]	Loss: 37.500000
Train Epoch: 50 [9600/18743 (51%)]	Loss: 62.500000
Train Epoch: 50 [9920/18743 (53%)]	Loss: 37.500000
Train Epoch: 50 [10240/18743 (55%)]	Loss: 43.750000
Train Epoch: 50 [10560/18743 (56%)]	Loss: 50.000000
Train Epoch: 50 [10880/18743 (58%)]	Loss: 50.000000
Train Epoch: 50 [11200/18743 (60%)]	Loss: 37.500000
Train Epoch: 50 [11520/18743 (61%)]	Loss: 62.500000
Train Epoch: 50 [11840/18743 (63%)]	Loss: 50.000000
Train Epoch: 50 [12160/18743 (65%)]	Loss: 56.250000
Train Epoch: 50 [12480/1

In [28]:
print("torch.cuda.memory_allocated: %fMB"%(torch.cuda.memory_allocated(0)/1024/1024))
print("torch.cuda.memory_reserved: %fMB"%(torch.cuda.memory_reserved(0)/1024/1024))
print("torch.cuda.max_memory_reserved: %fMB"%(torch.cuda.max_memory_reserved(0)/1024/1024))


torch.cuda.memory_allocated: 166.925293MB
torch.cuda.memory_reserved: 230.000000MB
torch.cuda.max_memory_reserved: 230.000000MB


### ToDo

- Check why test validation is not working
- Check how accuracy is calculated
- Create a section in which the network is described
- Model is not converging, chech why